In [12]:
# example of extracting and resizing faces into a new dataset
from os import listdir
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from numpy import asarray
from numpy import savez_compressed
from PIL import Image
from mtcnn.mtcnn import MTCNN
from matplotlib import pyplot

# load an image as an rgb numpy array
def load_image(filename):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = asarray(image)
    return pixels

# extract the face from a loaded image and resize
def extract_face(model, pixels, required_size=(80, 80)):
    # detect face in the image
    faces = model.detect_faces(pixels)
    # skip cases where we could not detect a face
    if len(faces) == 0:
        return None
    # extract details of the face
    x1, y1, width, height = faces[0]['box']
    # force detected pixel values to be positive (bug fix)
    x1, y1 = abs(x1), abs(y1)
    # convert into coordinates
    x2, y2 = x1 + width, y1 + height
    # retrieve face pixels
    face_pixels = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face_pixels)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

# load images and extract faces for all images in a directory
def load_faces(directory, n_faces):
    # prepare model
    model = MTCNN()
    faces = list()
    labels = []
    # enumerate files
    for filename in listdir(directory):
        print(filename)
        # load the image
        pixels = load_image(directory + filename)
        # get face
        face = extract_face(model, pixels)
        if face is None:
            continue
        # store
        faces.append(face)
        labels.append(filename)
        print(len(faces), face.shape)
        # stop once we have enough
        if len(faces) >= n_faces:
            break
    return asarray(faces), asarray(labels)

if __name__=='__main__':

    # directory that contains all images
    directory = '/git-repos/latent-space-arithmetic/dataset/img_align_celeba/'
    # load and extract all faces
    all_faces, face_labels = load_faces(directory, 10000)
    print('Loaded: ', all_faces.shape)
    # save in compressed format
    savez_compressed('img_align_celeba.npz', all_faces)
    savez_compressed('img_align_celeba_labels.npz', face_labels)

000001.jpg
1 (80, 80, 3)
000002.jpg
2 (80, 80, 3)
000003.jpg
3 (80, 80, 3)
000004.jpg
4 (80, 80, 3)
000005.jpg
5 (80, 80, 3)
000006.jpg
6 (80, 80, 3)
000007.jpg
7 (80, 80, 3)
000008.jpg
8 (80, 80, 3)
000009.jpg
9 (80, 80, 3)
000010.jpg
10 (80, 80, 3)
000011.jpg
11 (80, 80, 3)
000012.jpg
12 (80, 80, 3)
000013.jpg
13 (80, 80, 3)
000014.jpg
14 (80, 80, 3)
000015.jpg
15 (80, 80, 3)
000016.jpg
16 (80, 80, 3)
000017.jpg
17 (80, 80, 3)
000018.jpg
18 (80, 80, 3)
000019.jpg
19 (80, 80, 3)
000020.jpg
20 (80, 80, 3)
000021.jpg
21 (80, 80, 3)
000022.jpg
22 (80, 80, 3)
000023.jpg
23 (80, 80, 3)
000024.jpg
24 (80, 80, 3)
000025.jpg
25 (80, 80, 3)
000026.jpg
26 (80, 80, 3)
000027.jpg
27 (80, 80, 3)
000028.jpg
28 (80, 80, 3)
000029.jpg
29 (80, 80, 3)
000030.jpg
30 (80, 80, 3)
000031.jpg
31 (80, 80, 3)
000032.jpg
32 (80, 80, 3)
000033.jpg
33 (80, 80, 3)
000034.jpg
34 (80, 80, 3)
000035.jpg
35 (80, 80, 3)
000036.jpg
36 (80, 80, 3)
000037.jpg
37 (80, 80, 3)
000038.jpg
38 (80, 80, 3)
000039.jpg
39 (80, 80

251 (80, 80, 3)
000254.jpg
252 (80, 80, 3)
000255.jpg
253 (80, 80, 3)
000256.jpg
254 (80, 80, 3)
000257.jpg
255 (80, 80, 3)
000258.jpg
256 (80, 80, 3)
000259.jpg
257 (80, 80, 3)
000260.jpg
258 (80, 80, 3)
000261.jpg
259 (80, 80, 3)
000262.jpg
260 (80, 80, 3)
000263.jpg
261 (80, 80, 3)
000264.jpg
262 (80, 80, 3)
000265.jpg
263 (80, 80, 3)
000266.jpg
264 (80, 80, 3)
000267.jpg
265 (80, 80, 3)
000268.jpg
266 (80, 80, 3)
000269.jpg
267 (80, 80, 3)
000270.jpg
268 (80, 80, 3)
000271.jpg
269 (80, 80, 3)
000272.jpg
270 (80, 80, 3)
000273.jpg
271 (80, 80, 3)
000274.jpg
272 (80, 80, 3)
000275.jpg
273 (80, 80, 3)
000276.jpg
274 (80, 80, 3)
000277.jpg
275 (80, 80, 3)
000278.jpg
276 (80, 80, 3)
000279.jpg
277 (80, 80, 3)
000280.jpg
278 (80, 80, 3)
000281.jpg
279 (80, 80, 3)
000282.jpg
280 (80, 80, 3)
000283.jpg
281 (80, 80, 3)
000284.jpg
282 (80, 80, 3)
000285.jpg
283 (80, 80, 3)
000286.jpg
284 (80, 80, 3)
000287.jpg
285 (80, 80, 3)
000288.jpg
286 (80, 80, 3)
000289.jpg
287 (80, 80, 3)
000290.jpg
2

555 (80, 80, 3)
000558.jpg
556 (80, 80, 3)
000559.jpg
557 (80, 80, 3)
000560.jpg
558 (80, 80, 3)
000561.jpg
559 (80, 80, 3)
000562.jpg
560 (80, 80, 3)
000563.jpg
561 (80, 80, 3)
000564.jpg
562 (80, 80, 3)
000565.jpg
563 (80, 80, 3)
000566.jpg
564 (80, 80, 3)
000567.jpg
565 (80, 80, 3)
000568.jpg
566 (80, 80, 3)
000569.jpg
567 (80, 80, 3)
000570.jpg
568 (80, 80, 3)
000571.jpg
569 (80, 80, 3)
000572.jpg
570 (80, 80, 3)
000573.jpg
571 (80, 80, 3)
000574.jpg
572 (80, 80, 3)
000575.jpg
573 (80, 80, 3)
000576.jpg
574 (80, 80, 3)
000577.jpg
575 (80, 80, 3)
000578.jpg
576 (80, 80, 3)
000579.jpg
577 (80, 80, 3)
000580.jpg
578 (80, 80, 3)
000581.jpg
579 (80, 80, 3)
000582.jpg
580 (80, 80, 3)
000583.jpg
581 (80, 80, 3)
000584.jpg
582 (80, 80, 3)
000585.jpg
583 (80, 80, 3)
000586.jpg
584 (80, 80, 3)
000587.jpg
585 (80, 80, 3)
000588.jpg
586 (80, 80, 3)
000589.jpg
587 (80, 80, 3)
000590.jpg
588 (80, 80, 3)
000591.jpg
589 (80, 80, 3)
000592.jpg
590 (80, 80, 3)
000593.jpg
591 (80, 80, 3)
000594.jpg
5

859 (80, 80, 3)
000863.jpg
860 (80, 80, 3)
000864.jpg
861 (80, 80, 3)
000865.jpg
862 (80, 80, 3)
000866.jpg
863 (80, 80, 3)
000867.jpg
864 (80, 80, 3)
000868.jpg
865 (80, 80, 3)
000869.jpg
866 (80, 80, 3)
000870.jpg
867 (80, 80, 3)
000871.jpg
868 (80, 80, 3)
000872.jpg
869 (80, 80, 3)
000873.jpg
870 (80, 80, 3)
000874.jpg
871 (80, 80, 3)
000875.jpg
872 (80, 80, 3)
000876.jpg
873 (80, 80, 3)
000877.jpg
874 (80, 80, 3)
000878.jpg
875 (80, 80, 3)
000879.jpg
876 (80, 80, 3)
000880.jpg
877 (80, 80, 3)
000881.jpg
878 (80, 80, 3)
000882.jpg
879 (80, 80, 3)
000883.jpg
880 (80, 80, 3)
000884.jpg
881 (80, 80, 3)
000885.jpg
882 (80, 80, 3)
000886.jpg
883 (80, 80, 3)
000887.jpg
884 (80, 80, 3)
000888.jpg
885 (80, 80, 3)
000889.jpg
886 (80, 80, 3)
000890.jpg
887 (80, 80, 3)
000891.jpg
888 (80, 80, 3)
000892.jpg
889 (80, 80, 3)
000893.jpg
890 (80, 80, 3)
000894.jpg
891 (80, 80, 3)
000895.jpg
892 (80, 80, 3)
000896.jpg
893 (80, 80, 3)
000897.jpg
894 (80, 80, 3)
000898.jpg
895 (80, 80, 3)
000899.jpg
8

1157 (80, 80, 3)
001161.jpg
1158 (80, 80, 3)
001162.jpg
1159 (80, 80, 3)
001163.jpg
1160 (80, 80, 3)
001164.jpg
1161 (80, 80, 3)
001165.jpg
1162 (80, 80, 3)
001166.jpg
1163 (80, 80, 3)
001167.jpg
1164 (80, 80, 3)
001168.jpg
1165 (80, 80, 3)
001169.jpg
1166 (80, 80, 3)
001170.jpg
1167 (80, 80, 3)
001171.jpg
1168 (80, 80, 3)
001172.jpg
1169 (80, 80, 3)
001173.jpg
1170 (80, 80, 3)
001174.jpg
1171 (80, 80, 3)
001175.jpg
1172 (80, 80, 3)
001176.jpg
1173 (80, 80, 3)
001177.jpg
1174 (80, 80, 3)
001178.jpg
1175 (80, 80, 3)
001179.jpg
1176 (80, 80, 3)
001180.jpg
1177 (80, 80, 3)
001181.jpg
1178 (80, 80, 3)
001182.jpg
1179 (80, 80, 3)
001183.jpg
1180 (80, 80, 3)
001184.jpg
1181 (80, 80, 3)
001185.jpg
1182 (80, 80, 3)
001186.jpg
1183 (80, 80, 3)
001187.jpg
1184 (80, 80, 3)
001188.jpg
1185 (80, 80, 3)
001189.jpg
1186 (80, 80, 3)
001190.jpg
1187 (80, 80, 3)
001191.jpg
1188 (80, 80, 3)
001192.jpg
1189 (80, 80, 3)
001193.jpg
1190 (80, 80, 3)
001194.jpg
1191 (80, 80, 3)
001195.jpg
1192 (80, 80, 3)
001

...

8774 (80, 80, 3)
008789.jpg
8775 (80, 80, 3)
008790.jpg
8776 (80, 80, 3)
008791.jpg
8777 (80, 80, 3)
008792.jpg
8778 (80, 80, 3)
008793.jpg
8779 (80, 80, 3)
008794.jpg
8780 (80, 80, 3)
008795.jpg
8781 (80, 80, 3)
008796.jpg
8782 (80, 80, 3)
008797.jpg
8783 (80, 80, 3)
008798.jpg
8784 (80, 80, 3)
008799.jpg
8785 (80, 80, 3)
008800.jpg
8786 (80, 80, 3)
008801.jpg
8787 (80, 80, 3)
008802.jpg
8788 (80, 80, 3)
008803.jpg
8789 (80, 80, 3)
008804.jpg
8790 (80, 80, 3)
008805.jpg
8791 (80, 80, 3)
008806.jpg
8792 (80, 80, 3)
008807.jpg
8793 (80, 80, 3)
008808.jpg
8794 (80, 80, 3)
008809.jpg
8795 (80, 80, 3)
008810.jpg
8796 (80, 80, 3)
008811.jpg
8797 (80, 80, 3)
008812.jpg
8798 (80, 80, 3)
008813.jpg
8799 (80, 80, 3)
008814.jpg
8800 (80, 80, 3)
008815.jpg
8801 (80, 80, 3)
008816.jpg
8802 (80, 80, 3)
008817.jpg
8803 (80, 80, 3)
008818.jpg
8804 (80, 80, 3)
008819.jpg
8805 (80, 80, 3)
008820.jpg
8806 (80, 80, 3)
008821.jpg
8807 (80, 80, 3)
008822.jpg
8808 (80, 80, 3)
008823.jpg
8809 (80, 80, 3)
008

9067 (80, 80, 3)
009082.jpg
9068 (80, 80, 3)
009083.jpg
9069 (80, 80, 3)
009084.jpg
9070 (80, 80, 3)
009085.jpg
9071 (80, 80, 3)
009086.jpg
9072 (80, 80, 3)
009087.jpg
9073 (80, 80, 3)
009088.jpg
9074 (80, 80, 3)
009089.jpg
9075 (80, 80, 3)
009090.jpg
9076 (80, 80, 3)
009091.jpg
9077 (80, 80, 3)
009092.jpg
9078 (80, 80, 3)
009093.jpg
9079 (80, 80, 3)
009094.jpg
9080 (80, 80, 3)
009095.jpg
9081 (80, 80, 3)
009096.jpg
9082 (80, 80, 3)
009097.jpg
9083 (80, 80, 3)
009098.jpg
9084 (80, 80, 3)
009099.jpg
9085 (80, 80, 3)
009100.jpg
9086 (80, 80, 3)
009101.jpg
9087 (80, 80, 3)
009102.jpg
9088 (80, 80, 3)
009103.jpg
9089 (80, 80, 3)
009104.jpg
9090 (80, 80, 3)
009105.jpg
9091 (80, 80, 3)
009106.jpg
9092 (80, 80, 3)
009107.jpg
9093 (80, 80, 3)
009108.jpg
9094 (80, 80, 3)
009109.jpg
9095 (80, 80, 3)
009110.jpg
9096 (80, 80, 3)
009111.jpg
9097 (80, 80, 3)
009112.jpg
9098 (80, 80, 3)
009113.jpg
9099 (80, 80, 3)
009114.jpg
9100 (80, 80, 3)
009115.jpg
9101 (80, 80, 3)
009116.jpg
9102 (80, 80, 3)
009

9360 (80, 80, 3)
009375.jpg
9361 (80, 80, 3)
009376.jpg
9362 (80, 80, 3)
009377.jpg
9363 (80, 80, 3)
009378.jpg
9364 (80, 80, 3)
009379.jpg
9365 (80, 80, 3)
009380.jpg
9366 (80, 80, 3)
009381.jpg
9367 (80, 80, 3)
009382.jpg
9368 (80, 80, 3)
009383.jpg
9369 (80, 80, 3)
009384.jpg
9370 (80, 80, 3)
009385.jpg
9371 (80, 80, 3)
009386.jpg
9372 (80, 80, 3)
009387.jpg
9373 (80, 80, 3)
009388.jpg
9374 (80, 80, 3)
009389.jpg
9375 (80, 80, 3)
009390.jpg
9376 (80, 80, 3)
009391.jpg
9377 (80, 80, 3)
009392.jpg
9378 (80, 80, 3)
009393.jpg
9379 (80, 80, 3)
009394.jpg
9380 (80, 80, 3)
009395.jpg
9381 (80, 80, 3)
009396.jpg
9382 (80, 80, 3)
009397.jpg
9383 (80, 80, 3)
009398.jpg
9384 (80, 80, 3)
009399.jpg
9385 (80, 80, 3)
009400.jpg
9386 (80, 80, 3)
009401.jpg
9387 (80, 80, 3)
009402.jpg
9388 (80, 80, 3)
009403.jpg
9389 (80, 80, 3)
009404.jpg
9390 (80, 80, 3)
009405.jpg
9391 (80, 80, 3)
009406.jpg
9392 (80, 80, 3)
009407.jpg
9393 (80, 80, 3)
009408.jpg
9394 (80, 80, 3)
009409.jpg
9395 (80, 80, 3)
009

9653 (80, 80, 3)
009668.jpg
9654 (80, 80, 3)
009669.jpg
9655 (80, 80, 3)
009670.jpg
9656 (80, 80, 3)
009671.jpg
9657 (80, 80, 3)
009672.jpg
9658 (80, 80, 3)
009673.jpg
9659 (80, 80, 3)
009674.jpg
9660 (80, 80, 3)
009675.jpg
9661 (80, 80, 3)
009676.jpg
9662 (80, 80, 3)
009677.jpg
9663 (80, 80, 3)
009678.jpg
9664 (80, 80, 3)
009679.jpg
9665 (80, 80, 3)
009680.jpg
9666 (80, 80, 3)
009681.jpg
9667 (80, 80, 3)
009682.jpg
9668 (80, 80, 3)
009683.jpg
9669 (80, 80, 3)
009684.jpg
9670 (80, 80, 3)
009685.jpg
9671 (80, 80, 3)
009686.jpg
9672 (80, 80, 3)
009687.jpg
9673 (80, 80, 3)
009688.jpg
9674 (80, 80, 3)
009689.jpg
9675 (80, 80, 3)
009690.jpg
9676 (80, 80, 3)
009691.jpg
9677 (80, 80, 3)
009692.jpg
9678 (80, 80, 3)
009693.jpg
9679 (80, 80, 3)
009694.jpg
9680 (80, 80, 3)
009695.jpg
9681 (80, 80, 3)
009696.jpg
9682 (80, 80, 3)
009697.jpg
9683 (80, 80, 3)
009698.jpg
9684 (80, 80, 3)
009699.jpg
9685 (80, 80, 3)
009700.jpg
9686 (80, 80, 3)
009701.jpg
9687 (80, 80, 3)
009702.jpg
9688 (80, 80, 3)
009

9946 (80, 80, 3)
009961.jpg
9947 (80, 80, 3)
009962.jpg
9948 (80, 80, 3)
009963.jpg
9949 (80, 80, 3)
009964.jpg
9950 (80, 80, 3)
009965.jpg
9951 (80, 80, 3)
009966.jpg
9952 (80, 80, 3)
009967.jpg
9953 (80, 80, 3)
009968.jpg
9954 (80, 80, 3)
009969.jpg
9955 (80, 80, 3)
009970.jpg
9956 (80, 80, 3)
009971.jpg
9957 (80, 80, 3)
009972.jpg
9958 (80, 80, 3)
009973.jpg
9959 (80, 80, 3)
009974.jpg
9960 (80, 80, 3)
009975.jpg
9961 (80, 80, 3)
009976.jpg
9962 (80, 80, 3)
009977.jpg
9963 (80, 80, 3)
009978.jpg
9964 (80, 80, 3)
009979.jpg
9965 (80, 80, 3)
009980.jpg
9966 (80, 80, 3)
009981.jpg
9967 (80, 80, 3)
009982.jpg
9968 (80, 80, 3)
009983.jpg
9969 (80, 80, 3)
009984.jpg
9970 (80, 80, 3)
009985.jpg
9971 (80, 80, 3)
009986.jpg
9972 (80, 80, 3)
009987.jpg
9973 (80, 80, 3)
009988.jpg
9974 (80, 80, 3)
009989.jpg
9975 (80, 80, 3)
009990.jpg
9976 (80, 80, 3)
009991.jpg
9977 (80, 80, 3)
009992.jpg
9978 (80, 80, 3)
009993.jpg
9979 (80, 80, 3)
009994.jpg
9980 (80, 80, 3)
009995.jpg
9981 (80, 80, 3)
009

In [5]:
# # load the prepared dataset
# from numpy import load
# # load the face dataset
# data = load('img_align_celeba.npz')
# faces = data['arr_0']
# print('Loaded: ', faces.shape)

Loaded:  (1000, 80, 80, 3)
